In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated_umap.csv"
).resolve()
# shiny output path
shiny_output_path = pathlib.Path(
    "../temporal_shiny_app/CLS_features_annotated_umap.csv"
).resolve()

output_path.parent.mkdir(parents=True, exist_ok=True)
shiny_output_path.parent.mkdir(parents=True, exist_ok=True)

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_original_index,Metadata_compound,Metadata_dose,Metadata_control,...,channel_DNA_cls_feature_90,channel_DNA_cls_feature_91,channel_DNA_cls_feature_92,channel_DNA_cls_feature_93,channel_DNA_cls_feature_94,channel_DNA_cls_feature_95,channel_DNA_cls_feature_96,channel_DNA_cls_feature_97,channel_DNA_cls_feature_98,channel_DNA_cls_feature_99
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102,111256,Staurosporine,0.0,negative,...,0.034336,-0.005963,-0.032243,0.056527,-0.082777,0.036178,0.009163,0.018071,0.038674,-0.015397
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106,111257,Staurosporine,0.0,negative,...,0.008716,-0.049256,-0.029706,0.013455,-0.090113,0.029853,0.014054,-0.042019,0.045846,-0.025033
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,121,111258,Staurosporine,0.0,negative,...,0.016367,0.026408,-0.000336,0.040077,-0.120304,0.012534,0.008838,0.048772,0.048959,0.016963
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,127,111259,Staurosporine,0.0,negative,...,-0.009971,-0.015348,-0.035576,0.037665,-0.102958,-0.030505,0.026679,-0.000865,0.092421,-0.001723
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13,111241,Staurosporine,0.0,negative,...,0.016587,-0.023263,-0.012859,0.031189,-0.064073,0.017178,0.006649,0.058667,0.051056,-0.021589


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (188065, 10)
feature_df shape: (188065, 1536)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniforge3/envs/timelapse_env/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_original_index,Metadata_compound,Metadata_dose,Metadata_control,UMAP1,UMAP2
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102,111256,Staurosporine,0.0,negative,0.807103,4.956420
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106,111257,Staurosporine,0.0,negative,1.417984,4.481785
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,121,111258,Staurosporine,0.0,negative,1.491850,5.010065
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,127,111259,Staurosporine,0.0,negative,-2.599025,3.443170
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13,111241,Staurosporine,0.0,negative,2.953780,3.543933


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)

# save the UMAP embeddings to shiny app
umap_embedding_df.to_csv(shiny_output_path)